Copyright (c) 2020. Cognitive Scale Inc. All rights reserved.
Licensed under CognitiveScale Example Code [License](https://github.com/CognitiveScale/cortex-certifai-examples/blob/7998b8a481fccd467463deb1fc46d19622079b0e/LICENSE.md)

# Introduction

This fifth notebook shows you how to explore the trust score results in a notebook. If you have not already done so, please run the [fourth notebook](patient-readmission-trust-scan.ipynb) to scan the models for their trust scores.

In this notebook, we will:
1. Load the previously saved trust score reports
2. Convert the results into a dataframe and display them


In [1]:
import numpy as np
import pandas as pd
from pprint import pprint

from certifai.scanner.report_reader import ScanReportReader
from certifai.scanner.report_utils import scores, construct_scores_dataframe
from certifai.scanner.builder import ExplanationType
from IPython.display import display, Markdown

# Loading the Trust Score Reports

To load the reports, we need to know the use case ID ('readmission') and the scan ID.

List the available use cases, and the scans within the 'readmission' use case.

In [2]:
reader = ScanReportReader("reports")
reader.list_usecases()
scans = reader.list_scans('readmission')
data=[[s['date'], ', '.join(s['reportTypes']), s['id']] for s in scans]
df = pd.DataFrame(data, columns=['date', 'evals', 'scan id']).sort_values(by=['date'], ascending=False)
print(df)

              date                                      evals       scan id
0  20201102T134851  explainability, fairness, atx, robustness  eafa5614d892
1  20201102T131236                                explanation  c0e07952e6d6
2  20201102T103533                                explanation  df7a42f2ab84
3  20201102T102839                                explanation  b6de685a2712
4  20201016T132038                                explanation  5683ba38b998
5  20201011T151543  atx, fairness, robustness, explainability  4b13c55a4131
6  20201011T141903                                explanation  7a2e8b64cbd6


In [3]:
latest_scan = df[df['evals'].str.contains('fairness') & df['evals'].str.contains('robustness')].iloc[0]
results = reader.load_scan('readmission', latest_scan['scan id'])

# Extract the results

In this section we'll construct dataframes containing the score results and display them.

Print out the fairness results.

In [4]:
df = construct_scores_dataframe(scores('fairness', results), include_confidence=False)
display(df)

,context,type in [],fairness,Feature (age),type in Feature (age),Group details (15),type in Feature (age),Group details (25),type in Feature (age),Group details (35),...,type in Feature (race),Group details (Asian),type in Feature (race),Group details (Caucasian),type in Feature (race),Group details (Hispanic),type in Feature (race),Group details (Other),type in Feature (race),Group details (nan)
mlp (burden),mlp,burden,71.720752,75.494891,burden,0.056508,burden,0.132073,burden,0.087689,...,burden,0.036425,burden,0.086156,burden,0.080362,burden,0.064237,burden,0.017340
logit (burden),logit,burden,59.551743,59.686611,burden,0.031334,burden,0.218283,burden,0.101127,...,burden,0.029677,burden,0.100519,burden,0.075701,burden,0.054516,burden,0.017773


For an example of visualizing the fairness results in a chart, see the [fairness metrics notebook](https://raw.githubusercontent.com/CognitiveScale/cortex-certifai-examples/sdk-model-examples/notebooks/fairness_metrics).

Print out the robustness scores.

In [5]:
df = construct_scores_dataframe(scores('robustness', results), include_confidence=True)
display(df)

,context,robustness,robustness lower bound,robustness upper bound
mlp,mlp,70.003655,65.077138,74.713251
logit,logit,80.816004,76.619526,84.575458


Print out the explainability scores.

In [6]:
df = construct_scores_dataframe(scores('explainability', results), include_confidence=False)
display(df)

,context,explainability,Num features (1),Num features (10),Num features (2),Num features (3),Num features (4),Num features (5),Num features (6),Num features (7),Num features (8),Num features (9)
mlp,mlp,97.265625,75.78125,0.0,23.4375,0.00000,0.78125,0.0,0.0,0.0,0.0,0.0
logit,logit,97.617188,77.34375,0.0,21.8750,0.78125,0.00000,0.0,0.0,0.0,0.0,0.0
